In [16]:
import time
import numpy as np
import pandas as pd
import lightgbm as lgb
from functools import partial
import optuna
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

use_stack = False
new_study = False
do_lda = False
fix_data_skew = False

In [17]:
train_df = pd.read_csv('input/train' + ('_stack' if use_stack else '') + '.csv')
test_df = pd.read_csv('input/test'  + ('_stack' if use_stack else '') + '.csv')

if fix_data_skew:
    trues = train_df.loc[train_df['target'] == 1]
    falses = train_df.loc[train_df['target'] != 1].sample(frac=1)[:len(trues)]
    train_df = pd.concat([trues, falses], ignore_index=True).sample(frac=1)
else:
    train_df = train_df
    
train_df.head()

X_test = test_df.drop('ID_code',axis=1)
X = train_df.drop(['ID_code','target'],axis=1)
y = train_df['target']

In [18]:
if do_lda:    
    lda = LDA(solver='svd', n_components=5, store_covariance=True)
    X_lda = pd.DataFrame(lda.fit_transform(X, y))
    X_test_lda = pd.DataFrame(lda.transform(X_test))
    X["lda"] = X_lda
    X_test["lda"] = X_test_lda

In [19]:
n_splits = 5
folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

def objective(X, y, trial):
    score = 0.0

    # based on kernel params
    params = {
        "objective" : "binary",
        "metric" : "auc",
        "boosting": 'gbdt',
        "max_depth" : -1,
        "num_leaves" : trial.suggest_int('num_leaves', 13, 22), #13
        "learning_rate" : 0.01,
        "bagging_freq": trial.suggest_int('bagging_freq', 3, 7), #5
        "bagging_fraction" : trial.suggest_uniform('bagging_fraction', 0.3, 0.7), #0.4
        "feature_fraction" : trial.suggest_uniform('feature_fraction', 0.02, 0.06), #0.05
        "min_data_in_leaf": trial.suggest_int('min_data_in_leaf', 50, 80), #80
        "min_sum_heassian_in_leaf": trial.suggest_int('min_sum_heassian_in_leaf', 9, 15), #10
        "tree_learner": "serial",
        "boost_from_average": "false",
        #"lambda_l1" : 5,
        #"lambda_l2" : 5,
        "bagging_seed" : 13,
        "verbosity" : 1,
        "seed": 42
    }

    for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
        train_data = lgb.Dataset(X_train, label=y_train)
        valid_data = lgb.Dataset(X_valid, label=y_valid)
            
        model = lgb.train(params,train_data,num_boost_round=20000,
                        valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds=100)
        score += model.best_score['valid_1']['auc'] / n_splits
    print(score)
    return 1.0 - score

In [20]:
def main():
    f = partial(objective, X, y)
    if new_study:
        study = optuna.create_study(study_name='lgbm_kfold_study', storage='sqlite:///storage.db')
    else:
        study = optuna.Study(study_name='lgbm_kfold_study', storage='sqlite:///storage.db')
    study.optimize(f, n_trials=300)
    print('params:', study.best_params)

if __name__ == '__main__':
    main()

Fold 0 started at Fri Mar 29 18:26:11 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.894837	valid_1's auc: 0.871668
[600]	training's auc: 0.906644	valid_1's auc: 0.881376
[900]	training's auc: 0.912323	valid_1's auc: 0.885175
[1200]	training's auc: 0.916603	valid_1's auc: 0.887844
[1500]	training's auc: 0.920554	valid_1's auc: 0.890252
[1800]	training's auc: 0.924392	valid_1's auc: 0.892269
[2100]	training's auc: 0.927828	valid_1's auc: 0.893746
[2400]	training's auc: 0.931041	valid_1's auc: 0.894784
[2700]	training's auc: 0.9339	valid_1's auc: 0.895645
[3000]	training's auc: 0.936601	valid_1's auc: 0.896409
[3300]	training's auc: 0.939121	valid_1's auc: 0.897056
[3600]	training's auc: 0.9416	valid_1's auc: 0.897708
[3900]	training's auc: 0.943943	valid_1's auc: 0.898197
[4200]	training's auc: 0.946228	valid_1's auc: 0.898562
[4500]	training's auc: 0.948464	valid_1's auc: 0.898876
[4800]	training's auc: 0.950533	valid_1's auc: 0.899115
[5100

[I 2019-03-29 18:31:48,219] Finished a trial resulted in value: 0.10022903426378116. Current best value is 0.09980981822179502 with parameters: {'bagging_fraction': 0.4751059528094359, 'bagging_freq': 4, 'feature_fraction': 0.04027026162992961, 'min_data_in_leaf': 75, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 15}.


Fold 0 started at Fri Mar 29 18:31:49 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.893434	valid_1's auc: 0.872502
[600]	training's auc: 0.905405	valid_1's auc: 0.883125
[900]	training's auc: 0.911734	valid_1's auc: 0.887556
[1200]	training's auc: 0.915533	valid_1's auc: 0.891178
[1500]	training's auc: 0.918259	valid_1's auc: 0.892593
[1800]	training's auc: 0.920389	valid_1's auc: 0.893868
[2100]	training's auc: 0.922521	valid_1's auc: 0.8947
[2400]	training's auc: 0.924487	valid_1's auc: 0.895504
[2700]	training's auc: 0.926487	valid_1's auc: 0.896229
[3000]	training's auc: 0.928422	valid_1's auc: 0.896868
[3300]	training's auc: 0.930188	valid_1's auc: 0.897358
[3600]	training's auc: 0.931902	valid_1's auc: 0.89765
Early stopping, best iteration is:
[3756]	training's auc: 0.93275	valid_1's auc: 0.89787
Fold 1 started at Fri Mar 29 18:32:34 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.893379	va

[I 2019-03-29 18:36:23,697] Finished a trial resulted in value: 0.1008705848511342. Current best value is 0.09980981822179502 with parameters: {'bagging_fraction': 0.4751059528094359, 'bagging_freq': 4, 'feature_fraction': 0.04027026162992961, 'min_data_in_leaf': 75, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 15}.


Fold 0 started at Fri Mar 29 18:36:24 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.887611	valid_1's auc: 0.86987
[600]	training's auc: 0.899707	valid_1's auc: 0.87996
[900]	training's auc: 0.905678	valid_1's auc: 0.885382
[1200]	training's auc: 0.909555	valid_1's auc: 0.889226
[1500]	training's auc: 0.912316	valid_1's auc: 0.890746
[1800]	training's auc: 0.914524	valid_1's auc: 0.892354
[2100]	training's auc: 0.916603	valid_1's auc: 0.89352
[2400]	training's auc: 0.918448	valid_1's auc: 0.894539
[2700]	training's auc: 0.920463	valid_1's auc: 0.89544
[3000]	training's auc: 0.922268	valid_1's auc: 0.896104
[3300]	training's auc: 0.923936	valid_1's auc: 0.896689
[3600]	training's auc: 0.92552	valid_1's auc: 0.897081
[3900]	training's auc: 0.927091	valid_1's auc: 0.897563
[4200]	training's auc: 0.928495	valid_1's auc: 0.897982
[4500]	training's auc: 0.929866	valid_1's auc: 0.898318
[4800]	training's auc: 0.931209	valid_1's auc: 0.898552
[5100]

...

Fold 0 started at Sat Mar 30 20:29:10 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.898885	valid_1's auc: 0.873028
[600]	training's auc: 0.910466	valid_1's auc: 0.886598
[900]	training's auc: 0.916346	valid_1's auc: 0.890971
[1200]	training's auc: 0.920472	valid_1's auc: 0.893832
[1500]	training's auc: 0.923104	valid_1's auc: 0.895339
[1800]	training's auc: 0.925432	valid_1's auc: 0.896046
[2100]	training's auc: 0.927463	valid_1's auc: 0.896683
[2400]	training's auc: 0.929233	valid_1's auc: 0.89716
[2700]	training's auc: 0.930937	valid_1's auc: 0.897712
Early stopping, best iteration is:
[2683]	training's auc: 0.930876	valid_1's auc: 0.897754
Fold 1 started at Sat Mar 30 20:29:43 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.897947	valid_1's auc: 0.874032
[600]	training's auc: 0.910195	valid_1's auc: 0.885876
[900]	training's auc: 0.916491	valid_1's auc: 0.890864
[1200]	training's auc: 0.92059	v

[I 2019-03-30 20:31:41,728] Finished a trial resulted in value: 0.10250995034353005. Current best value is 0.09980981822179502 with parameters: {'bagging_fraction': 0.4751059528094359, 'bagging_freq': 4, 'feature_fraction': 0.04027026162992961, 'min_data_in_leaf': 75, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 15}.


Fold 0 started at Sat Mar 30 20:31:44 2019
Training until validation scores don't improve for 100 rounds.
[300]	training's auc: 0.893257	valid_1's auc: 0.872862
[600]	training's auc: 0.904469	valid_1's auc: 0.88487
[900]	training's auc: 0.910244	valid_1's auc: 0.888999
[1200]	training's auc: 0.914148	valid_1's auc: 0.892189
[1500]	training's auc: 0.91669	valid_1's auc: 0.89378
[1800]	training's auc: 0.918767	valid_1's auc: 0.894794
[2100]	training's auc: 0.920546	valid_1's auc: 0.895584
[2400]	training's auc: 0.922202	valid_1's auc: 0.896201
[2700]	training's auc: 0.923746	valid_1's auc: 0.89692
[3000]	training's auc: 0.925337	valid_1's auc: 0.897364
[3300]	training's auc: 0.926842	valid_1's auc: 0.897607
[3600]	training's auc: 0.928345	valid_1's auc: 0.898001
[3900]	training's auc: 0.929738	valid_1's auc: 0.898315
[4200]	training's auc: 0.931134	valid_1's auc: 0.898667
Early stopping, best iteration is:
[4383]	training's auc: 0.931948	valid_1's auc: 0.898858
Fold 1 started at Sat Mar 

[I 2019-03-30 20:36:31,627] Finished a trial resulted in value: 0.10056446100286498. Current best value is 0.09980981822179502 with parameters: {'bagging_fraction': 0.4751059528094359, 'bagging_freq': 4, 'feature_fraction': 0.04027026162992961, 'min_data_in_leaf': 75, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 15}.


params: {'bagging_fraction': 0.4751059528094359, 'bagging_freq': 4, 'feature_fraction': 0.04027026162992961, 'min_data_in_leaf': 75, 'min_sum_heassian_in_leaf': 12, 'num_leaves': 15}
